In [5]:
# USAGE
# python detect_face_parts.py --shape-predictor shape_predictor_68_face_landmarks.dat --image images/example_01.jpg 

# import the necessary packages
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
def fit_factor(dim,dis):
    fac = round(dis/dim,2)
    return fac
def map_factor(img_dim,s2,tar_dim):
    if tar_dim[1]>img_dim[0]-(s2+80):
        new_dim = img_dim[0]-(s2+30)
    else:
        new_dim = tar_dim[1]
    return new_dim  
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--shape-predictor", required=True,help="path to facial landmark predictor")
#ap.add_argument("-i", "--image", required=True,help="path to input image")
#args = vars(ap.parse_args())
cap = cv2.VideoCapture(0)
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
image1 = cv2.imread('neclace1.jpg') 
while True:
    ret, image = cap.read() 
    # load the input image, resize it, and convert it to grayscale
    #image = cv2.imread('gouse1.jpg')
    #print(image.shape)
    #image = imutils.resize(image, width=500)
    #print(image.shape)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over the face parts individually
        #for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
        name = 'jaw'
        (i, j) = face_utils.FACIAL_LANDMARKS_IDXS[name]
        (i, j) = (i+3,j-4)
        # clone the original image so we can draw on it, then
        # display the name of the face part on the image
        clone = image.copy()
        cv2.putText(clone,'neck', (10, 30), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 0, 255), 2)

        # loop over the subset of facial landmarks, drawing the
        # specific face part
        (s1,s2) = shape[i]
        (e1,e2) = shape[j]
        dis = e1-s1
        dim = image1.shape[1]
        print((s1,s2), (e1,e2))
        print(e1-s1)
        print(image1.shape)
        m_factor = fit_factor(dim,dis)
        p_dim = (int(image1.shape[1]*m_factor),int(image1.shape[0]*m_factor))
        print(p_dim)
        mf = map_factor(image.shape,s2,p_dim)
        if mf ==p_dim[0]:
            dim = p_dim
        else:
            dim = (p_dim[0],mf)
        print(dim)
        img2gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

        # add a threshold
        ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)
        #cv2.imshow('Ear Detector', ear_img) 
        mask_inv = cv2.bitwise_not(mask)
        resized = cv2.resize(image1,dim, interpolation = cv2.INTER_AREA)
        #print(resized.shape)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #print(mask.shape)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #print(mask_inv.shape)
        resized1 = cv2.resize(resized, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(mask, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(mask.shape)
        mask_inv = cv2.resize(mask_inv, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(mask_inv.shape)
        for (x, y) in shape[i:j]:
            cv2.circle(clone, (x, y+40), 3, (0, 255, 0), -1)

        # extract the ROI of the face region as a separate image
        #(x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        #roi = image[y:y + h, x:x + w]
        #roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        print(x, y, w, h)
        point = s1
        point1 = s2+30
        roi1 = image[point1:point1+dim[1],point:point+dim[0]]
        print(roi1.shape)
        print(point1+dim[1],point+dim[0])
        roi = cv2.resize(roi1, (roi1.shape[1]*10,roi1.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(s1,point,point1,point1+dim[0])
        print(roi.shape)
        print(point,point1)
        roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
        print(roi_bg.shape)
        # roi_fg contains the orignal location of earring
        roi_fg = cv2.bitwise_and(resized1,resized1,mask = mask)
        print(roi_bg.shape)
        # joining the roi_bg and roi_fg
        dst1 = cv2.add(roi_bg,roi_fg)
        dst = cv2.resize(dst1, (int(dst1.shape[1]/10),int(dst1.shape[0]/10)), interpolation = cv2.INTER_AREA)
        print(dst.shape)
        #cv2.imshow('Ear Detector', dst) 
        #cv2.waitKey(10)
        #point = int(s1+dim[0])
        #print(point,point+dim[0])
        cv2.circle(image, (s1,s2+80), 3, (0, 255, 0), -1)
        #cv2.rectangle(image, (s1,s2+80), (s1+dim[0],s2+80+dim[1]), (0,255,0), 3)
        image[point1:point1+dim[1],point:point+dim[0]]=dst
        # show the particular face part
        #cv2.imshow("ROI", roi)
        cv2.imshow("Image", image)
        #cv2.waitKey(0)
        # visualize all facial landmarks with a transparent overlay
        #output = face_utils.visualize_facial_landmarks(image, shape)
        #cv2.imshow("Image", output)
        #cv2.waitKey(0)
    k = cv2.waitKey(0) & 0xFF
    if k == ord('q'): # wait for ESC key to exit
        break
    #if cv2.waitKey(0) & 0xFF == ord('s'):
        #break

cap.release() 
cv2.destroyAllWindows()

(304, 323) (504, 330)
200
(847, 493, 3)
(202, 347)
(202, 127)
(1270, 2020)
(1270, 2020)
304 323 191 80
(127, 202, 3)
480 506
304 304 353 555
(1270, 2020, 3)
304 353
(1270, 2020, 3)
(1270, 2020, 3)
(127, 202, 3)


In [6]:
def fit_factor(dim,dis):
    fac = round(dis/dim,2)
    return fac

In [7]:
print(map_factor((480,640),240,(228,113)))

113


In [1]:
def map_factor(img_dim,s2,tar_dim):
    if tar_dim[0]>img_dim[0]-(s2+80):
        new_dim = img_dim[0]-(s2+30)
    else:
        new_dim = tar_dim[0]
    return new_dim       

In [2]:
# USAGE
# python detect_face_parts.py --shape-predictor shape_predictor_68_face_landmarks.dat --image images/example_01.jpg 

# import the necessary packages
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
import urllib.request
def fit_factor(dim,dis):
    fac = round(dis/dim,2)
    return fac
def map_factor(img_dim,s2,tar_dim):
    if tar_dim[1]>img_dim[0]-(s2+80):
        new_dim = img_dim[0]-(s2+10)
    else:
        new_dim = tar_dim[1]
    return new_dim  
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--shape-predictor", required=True,help="path to facial landmark predictor")
#ap.add_argument("-i", "--image", required=True,help="path to input image")
#args = vars(ap.parse_args())
cap = cv2.VideoCapture(0)
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
image1 = cv2.imread('neclace4.jpg') 
URL = "http://192.168.43.1:8080/shot.jpg"
while True:
    #ret, image = cap.read() 
    # load the input image, resize it, and convert it to grayscale
    #image = cv2.imread('gouse1.jpg')
    #print(image.shape)
    #image = imutils.resize(image, width=500)
    #print(image.shape)
    img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
    #print(img_arr)
    image = cv2.imdecode(img_arr,-1)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over the face parts individually
        #for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
        name = 'jaw'
        (i, j) = face_utils.FACIAL_LANDMARKS_IDXS[name]
        (i, j) = (i+3,j-4)
        # clone the original image so we can draw on it, then
        # display the name of the face part on the image
        clone = image.copy()
        cv2.putText(clone,'neck', (10, 30), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 0, 255), 2)

        # loop over the subset of facial landmarks, drawing the
        # specific face part
        (s1,s2) = shape[i]
        (e1,e2) = shape[j]
        dis = e1-s1
        dim = image1.shape[1]
        print((s1,s2), (e1,e2))
        print(e1-s1)
        print(image1.shape)
        m_factor = fit_factor(dim,dis)
        p_dim = (int(image1.shape[1]*m_factor),int(image1.shape[0]*m_factor))
        print(p_dim)
        mf = map_factor(image.shape,s2,p_dim)
        if mf ==p_dim[0]:
            dim = p_dim
        else:
            dim = (p_dim[0],mf)
        print(dim)
        img2gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

        # add a threshold
        ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)
        #cv2.imshow('Ear Detector', ear_img) 
        mask_inv = cv2.bitwise_not(mask)
        resized = cv2.resize(image1,dim, interpolation = cv2.INTER_AREA)
        #print(resized.shape)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #print(mask.shape)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #print(mask_inv.shape)
        resized1 = cv2.resize(resized, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(mask, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(mask.shape)
        mask_inv = cv2.resize(mask_inv, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(mask_inv.shape)
        for (x, y) in shape[i:j]:
            cv2.circle(clone, (x, y+40), 3, (0, 255, 0), -1)

        # extract the ROI of the face region as a separate image
        #(x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        #roi = image[y:y + h, x:x + w]
        #roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        print(x, y, w, h)
        point = s1
        point1 = s2+50
        roi1 = image[point1:point1+dim[1],point:point+dim[0]]
        print(roi1.shape)
        print(point1+dim[1],point+dim[0])
        roi = cv2.resize(roi1, (roi1.shape[1]*10,roi1.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(s1,point,point1,point1+dim[0])
        print(roi.shape)
        print(point,point1)
        roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
        print(roi_bg.shape)
        # roi_fg contains the orignal location of earring
        roi_fg = cv2.bitwise_and(resized1,resized1,mask = mask)
        print(roi_bg.shape)
        # joining the roi_bg and roi_fg
        dst1 = cv2.add(roi_bg,roi_fg)
        dst = cv2.resize(dst1, (int(dst1.shape[1]/10),int(dst1.shape[0]/10)), interpolation = cv2.INTER_AREA)
        print(dst.shape)
        #cv2.imshow('Ear Detector', dst) 
        #cv2.waitKey(10)
        #point = int(s1+dim[0])
        #print(point,point+dim[0])
        cv2.circle(image, (s1,s2+80), 3, (0, 255, 0), -1)
        #cv2.rectangle(image, (s1,s2+80), (s1+dim[0],s2+80+dim[1]), (0,255,0), 3)
        image[point1:point1+dim[1],point:point+dim[0]]=dst
        # show the particular face part
        #cv2.imshow("ROI", roi)
        cv2.imshow("Image", image)
        #cv2.waitKey(0)
        # visualize all facial landmarks with a transparent overlay
        #output = face_utils.visualize_facial_landmarks(image, shape)
        #cv2.imshow("Image", output)
        #cv2.waitKey(0)
    k = cv2.waitKey(0) & 0xFF
    if k == ord('q'): # wait for ESC key to exit
        break
    #if cv2.waitKey(0) & 0xFF == ord('s'):
        #break

cap.release() 
cv2.destroyAllWindows()

KeyboardInterrupt: 